In [21]:
%matplotlib auto
# %matplotlib auto
import numpy as np
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import cv2

import sys
import os.path
print(os.getcwd())
sys.path.insert(0, os.getcwd()+"/../")
from ContourTrackbarFilter import ContourTrackbarFilter
sys.path.insert(0, os.getcwd()+"/../../../../libs/tacx")
print(os.getcwd()+"/../../../../libs/tacx")
from SEMContour import *
from MxpStage import MxpStageXmlParser
sys.path.insert(0, os.getcwd()+"/../../../../libs/imutil")
from ImGUI import imread_gray
sys.path.insert(0, os.getcwd()+"/../../../../libs/common")
from FileUtil import gpfs2WinPath

CWD = r'D:\code\Python\apps\MXP\ContourSelect\samplejob\h\cache\dummydb\result\MXP\job1'
# CWD = r'C:\Localdata\D\Note\Python\apps\MXP\ContourSelect\samplejob\h\cache\dummydb\result\MXP\job1'
CWD = gpfs2WinPath(CWD)

inxml = os.path.join(CWD, r'contourlabeling610out.xml') # contourlabeling410out.xml

def loadPatternData(imgfile='', contourfile=''):
    bSucceedReadCt = False
    # read contour
    contour = SEMContour()
    bSucceedReadCt = contour.parseFile(contourfile)
    if not bSucceedReadCt:
        raise OSError("Error, contourfile('{}') cannot be parsed".format(contourfile))

    # read image
    try:
        im, _ = imread_gray(imgfile)
    except:
        im = None
        print('input image is none')
    return im, contour

def updateContourROI(contour, im=None, mode='crop', overlay=True):
    '''
    Update contour ROI, mode could be 'crop' or 'extend'

    Parameters
    ----------
    mode:   string
        * 'crop': crop the image into contour bbox, output `contour point cords -= (xini, yini)`
        * 'crop': extend the image into full size, output `contour point cords += (xini, yini)`
    overly: boolean
        * 'True': output image is the overlay of image + contour
        * 'False': output image is just image itself
    
    Returns
    -------
    outim: image object
        could be overlay of image + contour or image itself
    outcontour: SEMContour object
        contour file will different cords
    '''
    outim, outcontour = None, None
    # process image
    if im is None:
        print("input image is None, use gray background")
        imw, imh = contour.getshape()
        imw, imh = int(imw), int(imh)
        im = 255//2 * np.ones((imh, imw, 3), dtype=np.uint8) # gray background
    else:
        if len(im.shape) == 2:
            im = cv2.cvtColor(im, cv2.COLOR_GRAY2BGR)
        if not np.issubdtype(im.dtype, np.integer):
            raise TypeError("only support int type images, input image type: {}".format(im.dtype))

    vmax = np.iinfo(im.dtype).max
    CONTOUR_COLOR = (0, vmax, vmax) # yellow
    # CONTOUR_COLOR = (0, vmax, 0) # green

    # process contour, and overlay image
    df = contour.toDf()
    xini, yini, xend, yend = contour.getBBox()
    print(xini, yini, xend, yend)
    if mode == 'crop':
        outim = im[yini:yend, xini:xend]
        df.loc[:, 'offsetx'] = df.loc[:, 'offsetx'].sub(xini)
        df.loc[:, 'offsety'] = df.loc[:, 'offsetx'].sub(yini)
        if overlay:
            contourPointsVec = []
            grouped = df[['polygonId', 'offsetx', 'offsety']].groupby('polygonId')
            total = 0
            for name, group in grouped:
                points = group.loc[:, ['offsetx', 'offsety']].values.astype('int32')
                contourPointsVec.append(points)
                total += len(points)
            print("total points: {}".format(total))
            thickness = 1
            #print('contourPointsVec data shape', np.array(contourPointsVec).shape, contourPointsVec[0].shape)
            outim = cv2.polylines(outim, contourPointsVec, False, CONTOUR_COLOR, thickness)         
    elif mode == 'extend':
        df.loc[:, 'offsetx'] = df.loc[:, 'offsetx'].add(xini)
        df.loc[:, 'offsety'] = df.loc[:, 'offsetx'].add(yini)
    else:
        raise ValueError("Only support mode of 'crop' or 'extend', input is {}".format(mode))
    print("total df points: {}".format(len(df)))   
    outcontour = contour.fromDf(df)
    
    totalPointsNum = 0
    for i in range(len(outcontour.getPolygonData())):
        totalPointsNum += outcontour.getPolygonData()[i]['vertexNum']
    
    print("total outdf points: #point {} len(df) {}".format(totalPointsNum, len(outcontour.toDf())))
    outcontour.saveContour('test.txt')
    
    return outim, outcontour

def getContourClassifierData(inxml):
    print(inxml)
    ocf_parser = MxpStageXmlParser(inxml) #'inxml', 'outxml'
    # icf = icf_parser.icf
    df_ocf = ocf_parser.occfs2df()
    patternid = 461
    df_occf = df_ocf.loc[df_ocf.name==patternid, :]
    sr_occf = pd.Series(df_occf.values.flatten(), index=df_occf.columns)
    contourfile = getRealFilePath(sr_occf.loc['contour/path'])
    imgfile = getRealFilePath(sr_occf.loc['image/path'])
    #print(patternid, contourfile, imgfile)
    im, contour = loadPatternData(imgfile, contourfile)
    
    return im, contour
    # TODO, add bbox plot
    # df.filter(regex='bbox/outlier', axis=1)

def getRealFilePath(curfile):
    if '/' in curfile:
        ossep = '/'
    else:
        ossep = '\\'
    return os.path.join(CWD, os.sep.join(curfile.split(ossep)))

Using matplotlib backend: Qt5Agg
D:\code\Python\apps\MXP\ContourSelect\unittest
D:\code\Python\apps\MXP\ContourSelect\unittest/../../../../libs/tacx


In [ ]:
# plot by column unique labels
def plot_col_by_label(contour, patternid='', colname=''):
    df = contour.toDf()
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    xini, yini, xend, yend = contour.getBBox()
    ax.set_xlim([xini, xend])
    ax.set_ylim([yini, yend])
    ax.set_title("Pattern "+patternid)
    
    uniqVals = df.loc[:, colname].drop_duplicates().values
    print(uniqVals)
    for label in uniqVals:
        flt_eq = df.loc[:, colname] == label
        if label == 'nan':
            flt_eq = df.loc[:, colname].isna()
        ax.plot(df.loc[flt_eq, 'offsetx'], df.loc[flt_eq, 'offsety'], '.', linestyle='None',  markersize=2, label=colname+'=={}'.format(label))

    plt.gca().invert_yaxis()
    plt.legend()
    plt.show()

In [ ]:
# plot the SEM image, contour and angle
def plot_image_contour_angle(im, contour, patternid='', arrow_length=1):
    df = contour.toDf()
    fig = plt.figure()
    ax = fig.add_subplot(111)
    ax.set_aspect('equal')
    #imw, imh = contour.getshape()
    #ax.set_xlim([0, imw])
    #ax.set_ylim([0, imh])
    xini, yini, xend, yend = contour.getBBox()
    ax.set_xlim([xini, xend])
    ax.set_ylim([yini, yend])
    ax.set_title("Pattern "+patternid+ " image Contour")
    
    # plot image
    ax.imshow(im)
    
    # plot contour
    ax.plot(df.loc[:, 'offsetx'], df.loc[:, 'offsety'], 'b.')
    ax.plot(250.480209, 715.985352, 'r.')
    
    # plot angle
    for _, row in df.iterrows():
        x, y = row.loc['offsetx'], row.loc['offsety']
        angle = row.loc['angle']
        dx, dy = arrow_length*np.cos(angle), arrow_length*np.sin(angle)
        ax.arrow(x, y, dx, dy, width=0.1, fc='y', ec='y') # ,shape='right', overhang=0
        
    plt.gca().invert_yaxis()
    plt.show()

In [ ]:
# SEM Contour Selection resulst plot: by TP, FN, FP, TP
def plotContourClassifier(im, contour, wndname=''):
    # plot image and classified contour point
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    imw, imh = contour.getshape()
    '''
    ax.set_aspect('equal')
    ax.set_xlim([0, imw])
    ax.set_ylim([0, imh])
    '''
    xini, yini, xend, yend = contour.getBBox()
    ax.set_xlim([xini, xend])
    ax.set_ylim([yini, yend])
    ax.set_title(wndname)

    df = contour.toDf()
    TP = (df.UserLabel==0) & (df.ClfLabel==0)
    FN = (df.UserLabel==0) & (df.ClfLabel==1)
    FP = (df.UserLabel==1) & (df.ClfLabel==0)
    TN = (df.UserLabel==1) & (df.ClfLabel==1)
    
    # calculate confusion matrix
    cm = np.array([len(df.loc[flt, :]) for flt in [TP, FN, FP, TN]]).reshape((2, 2))
    cm_norm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
    
    ax.imshow(im)
    ax.plot(df.loc[TP ,'offsetx'], df.loc[TP, 'offsety'], #'b.', markersize=1, 
            linestyle='None', marker= 'o', markersize=2, markeredgewidth=1, markerfacecolor='none', 
            label='TP, UserLabel=0 & ClfLabel=0: {}({:.3f}%)'.format(cm[0, 0], cm_norm[0, 0]*100 ))
    ax.plot(df.loc[FN ,'offsetx'], df.loc[FN, 'offsety'],
            linestyle='None', marker= 'o', markersize=4, markeredgewidth=1, markerfacecolor='none', 
            label='FN, UserLabel=1 & ClfLabel=0: {}({:.3f}%)'.format(cm[0, 1], cm_norm[0, 1]*100 ))
    ax.plot(df.loc[FP ,'offsetx'], df.loc[FP, 'offsety'], 
            linestyle='None', marker= 'o', markersize=4, markeredgewidth=1, markerfacecolor='none', 
            label='FP, UserLabel=0 & ClfLabel=1: {}({:.3f}%)'.format(cm[1, 0], cm_norm[1, 0]*100 ))
    ax.plot(df.loc[TN ,'offsetx'], df.loc[TN, 'offsety'], #'r*', markersize=2,
            linestyle='None', marker= 'o', markersize=2, markeredgewidth=1, markerfacecolor='none', 
            label='TN, UserLabel=1 & ClfLabel=1: {}({:.3f}%)'.format(cm[1, 1], cm_norm[1, 1]*100 ))
    
    #ax = plt.gca() # gca() function returns the current Axes instance
    #ax.set_ylim(ax.get_ylim()[::-1]) # reverse Y
    plt.gca().invert_yaxis()
    plt.legend(loc=1)
    plt.show()
    
# SEM Contour Selection resulst plot: by classifer Positive 0, & Negative 1
def plotContourDiscriminator(im, contour, wndname=''):
    # plot image and classified contour point
    fig = plt.figure()
    ax = fig.add_subplot(111)
    
    imw, imh = contour.getshape()
    '''
    ax.set_aspect('equal')
    ax.set_xlim([0, imw])
    ax.set_ylim([0, imh])
    '''
    xini, yini, xend, yend = contour.getBBox()
    ax.set_xlim([xini, xend])
    ax.set_ylim([yini, yend])
    ax.set_title(wndname)

    df = contour.toDf()
    Positive = df.ClfLabel==0
    Negative = df.ClfLabel==1

    # calculate confusion matrix
    cm = np.array([len(df.loc[flt, :]) for flt in [Positive, Negative]])
    cm_norm = cm.astype('float') / cm.sum()
    
    ax.imshow(im)
    ax.plot(df.loc[Positive ,'offsetx'], df.loc[Positive, 'offsety'], #'b.', markersize=1, 
            linestyle='None', marker= 'o', markersize=2, markeredgewidth=1, markerfacecolor='none', 
            label='Discriminator Positive, ClfLabel=0: {}({:.3f}%)'.format(cm[0], cm_norm[0]*100 ))
    ax.plot(df.loc[Negative ,'offsetx'], df.loc[Negative, 'offsety'], #'r*', markersize=2,
            linestyle='None', marker= 'o', markersize=2, markeredgewidth=1, markerfacecolor='none', 
            label='Discriminator Negative, ClfLabel=1: {}({:.3f}%)'.format(cm[1], cm_norm[1]*100 ))
    
    #ax = plt.gca() # gca() function returns the current Axes instance
    #ax.set_ylim(ax.get_ylim()[::-1]) # reverse Y
    plt.gca().invert_yaxis()
    plt.legend(loc=1)
    plt.show()

In [ ]:
def plotAllContourClfData(inxml):
    ocf_parser = MxpStageXmlParser(inxml) #, 'outxml'
    # icf = icf_parser.icf
    df_ocf = ocf_parser.occfs2df()
    for _, row in df_ocf.iterrows():
        patternid = row.loc['name']
        usage = row.loc['usage']
        print("{} {}".format(patternid, usage))
        
        contourfile = row.loc['contour/path']
        contourfile = CWD + '/' + contourfile
        imgfile = row.loc['image/path']
        imgfile = CWD + '/' + imgfile
        im, contour = loadPatternData(imgfile, contourfile)
        plotContourClassifier(im, contour, "Pattern "+str(patternid))
        plotContourDiscriminator(im, contour, "Pattern "+str(patternid))

In [22]:
singlePatternPlot = 1
if singlePatternPlot:
    im, contour = getContourClassifierData(inxml)
    #plot_col_by_label(contour, patternid='461', colname="UserLabel")
    #plot_image_contour_angle(im, contour, '461')

    #plotContourClassifier(im, contour, 'Pattern 461')
    #plotContourDiscriminator(im, contour, 'Pattern 461')

    overlayim, biasedcontour = updateContourROI(contour, im, mode='crop', overlay=True)
    #drawer = ContourTrackbarFilter(overlayim, biasedcontour, 'Pattern 461')
    #thres = drawer.run_wi_row()
    #thres = drawer.run()
    #print(thres)
else:
    plotAllContourClfData(inxml)

D:\code\Python\apps\MXP\ContourSelect\samplejob\h\cache\dummydb\result\MXP\job1\contourlabeling610out.xml
bbox in origin image: [189, 189, 835, 835]
189 189 835 835
total points: 8187
total df points: 8187
total outdf points: #point 8187 len(df) 3536


In [24]:
print(overlayim.shape)
bdf = biasedcontour.toDf()
sr = bdf.loc[:6, 'offsetx']
print(sr)
sr = sr.sub(100)
print(sr)

(646, 646, 3)
0    380.618469
1    380.617310
2    380.228882
3    380.074036
4    379.688721
5    379.573853
6    379.890320
Name: offsetx, dtype: float64
0    280.618469
1    280.617310
2    280.228882
3    280.074036
4    279.688721
5    279.573853
6    279.890320
Name: offsetx, dtype: float64


In [6]:
print(im.shape)
contour.toDf().describe()

(1024, 1024)


,polygonId,offsetx,offsety,angle,weight,confidence,intensity,slope,band_width,ridge_intensity,curvature,contrast,mxp_flag,EigenRatio,UserLabel
count,8187.000000,8187.000000,8187.000000,8187.000000,8187.0,8187.000000,8187.000000,8187.000000,8187.0,8187.000000,8187.000000,8187.000000,8187.000000,8187.000000,53.000000
mean,28.324783,517.754837,511.725061,0.089124,1.0,0.865641,0.533704,-0.003241,0.0,0.018120,-0.108407,0.039546,0.418224,-0.025060,0.566038
std,16.143155,190.885731,185.592049,2.010000,0.0,0.341059,0.050327,0.008526,0.0,0.005060,0.319001,0.021738,2.552933,0.131941,0.500363
min,0.000000,195.518875,195.954636,-3.141574,1.0,0.000000,0.242484,-0.021712,0.0,0.000854,-1.000000,0.000204,0.000000,-0.999830,0.000000
25%,14.000000,352.644684,343.013046,-0.114466,1.0,1.000000,0.500713,-0.009493,0.0,0.015838,0.000788,0.021359,0.000000,-0.048495,0.000000
50%,27.000000,541.109436,512.001099,-0.000898,1.0,1.000000,0.560400,-0.008088,0.0,0.018628,0.002428,0.037620,0.000000,-0.004729,1.000000
75%,43.000000,685.278656,679.891937,0.263733,1.0,1.000000,0.572903,0.006411,0.0,0.020524,0.005024,0.056868,0.000000,0.033451,1.000000
max,61.000000,824.136108,827.051880,3.141556,1.0,1.000000,0.607019,0.020774,0.0,0.027109,0.154395,0.101304,16.000000,0.171571,1.000000
